# Initializing your environment

### Setting up a virtual environment
(Optional but recommended)

```bash
virtualenv -p python3 venv
source venv/bin/activate
```
(Use `deactivate` to exit from `source` once you are done)

Alternatively you can also prefix all your `python` and `pip` commands with `./venv/bin/` (e.g: `./venv/bin/pip3 install -U pip`)



## Setting up Jupyter

In order to follow along on your computer:

```bash
pip3 install notebook
jupyter-notebook
```

## Installation of PyMISP

#### Make sure the submodules are up-to-date and cloned

```bash
git submodule update --init
```

#### Install PyMISP with the developer options

```bash
cd PyMISP
pip3 install -e .
```

# Using the PyMISP objects

PyMISP is the python library used to deal with MISP format so you do not have to deal with the JSON blob yourself without knowing about the required and optional fields.

## MISPEvent

MISPEvent is the main class to use when you want to create/update events on a MISP instance.

In [ ]:
from pymisp import MISPEvent

event = MISPEvent()

event.info = 'A fancy MISP Event'  # Required
event.distribution = 0  # Optional, sets the distribution level to "Your Organisation only"
event.threat_level_id = 2  # Optional, sets the threat level to "Medium"
event.analysis = 1  # Optional, set the analysis to "Ongoing"

print(event.to_json(indent=4))

### Set the event date

The date can be in many different formats. PyMISP normalizes it in a way that will be understood by your MISP instance: a date in the `YYYY-MM-DD` format.

In [ ]:
event.set_date('2022-01-06')
print(f'From a text date: {event.date}')

from datetime import date
d = date.today()
event.set_date(d)
print(f'From a datetime.date date: {event.date}')

from datetime import datetime
d = datetime.now()
event.set_date(d)
# MISP expects a day, so the Hour will be dropped
print(f'From a datetime.datetime date: {event.date}')

### Tag an Event

An easy way to tag an Event

In [ ]:
event.add_tag('tlp:white')

print(event.to_json(indent=4))

## MISP Attribute

MISP Attributes are the raw pieces of data that can either be indicators of compromise (IoCs) or observed data.

They are defined with a triplet of required values being a `type`, a `category` and a `value`, and a bunch of optional fields.

With PyMISP, the Attributes parameters are the following:
- **type** (required)
- **value** (required)
- **category**: the type of information (if not set, the default one for the given `type` is used)
- **to_ids**: defines whether the attribute defines some malicious data that should be blocked, or not (if not set, the default value for the given `type` is used)
- **distribution**: defaults to inherit from parent (event or object)
- **disable_correlation**: if you want to avoid correlations between events on that specific value
- **data**: for `malware-sample` and `attachment` types, BytesIO object of the file.

### A minimal and quick way of adding an attribute

In [ ]:
attribute1 = event.add_attribute('ip-dst', '8.8.8.8')

print(attribute1.to_json(indent=4))

### Set inline parameters

In [ ]:
attribute2 = event.add_attribute('ip-dst', '8.8.8.9', disable_correlation=True)

print(attribute2.to_json(indent=4))

### Modify an existing attribute

Every parameter can be modified in a pythonic way

In [ ]:
attribute1.to_ids = False

print(attribute1.to_json(indent=4))

### Soft delete an attribute

The default approach on MISP is to soft delete data, which means it will not be displayed in the dafult view on MISP.  
The reason for doing this is to allow to push delete updates to instances we synchronise with.

In [ ]:
attribute2.delete()

print(event.to_json(indent=4))

### A more advanced way of passing the different parameters at once

In [ ]:
from uuid import uuid4

attribute_uuid = uuid4()
print(attribute_uuid)

kwargs = {
    'to_ids': False,
    'disable_correlation': True,
    'category': 'Network activity',
    'uuid': attribute_uuid
}
attribute = event.add_attribute('ip-src', '1.1.1.1', **kwargs)

print(attribute.to_json(indent=4))

### Using the MISPAttribute class

Allows you to play with the attribute before adding it to the event.

It is then possible to load the attribute from a JSON or from a dict

In [ ]:
from pymisp import MISPAttribute

attribute = MISPAttribute()
attribute.type = 'domain'
attribute.value = 'circl.lu'

print(attribute.to_json(indent=4))
print(event.add_attribute(**attribute).to_json(indent=4))

In [ ]:
# Loaded from a JSON
json = '''{
    "type": "domain",
    "value": "circl.lu",
    "to_ids": false
}'''

json_attribute = MISPAttribute()
json_attribute.from_json(json)

print(json_attribute.to_json(indent=4))

In [ ]:
# Loaded from a python dict
_dict = {
    'type': 'domain',
    'value': 'circl.lu',
    'to_ids': False
}

dict_attribute = MISPAttribute()
dict_attribute.from_dict(**_dict)

print(dict_attribute.to_json(indent=4))